In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exist-2024-clef/train_data_task1.csv
/kaggle/input/exist-2024-clef/dev_data_task3.csv
/kaggle/input/exist-2024-clef/train_data_task2.csv
/kaggle/input/exist-2024-clef/dev_data_task1.csv
/kaggle/input/exist-2024-clef/train_data_task3.csv
/kaggle/input/exist-2024-clef/EXIT2024_tweet_test.csv
/kaggle/input/exist-2024-clef/dev_data_task2.csv


In [ ]:
!rm -rf /kaggle/working/output/best-model/task2/hard/*

In [2]:
!wandb offline
!wandb disabled

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.
W&B disabled.


In [3]:
!pip install datasets
!pip install transformers

In [4]:
from  IPython. display import clear_output
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
import random
import  matplotlib. pyplot  as  plt
from tqdm import tqdm
import torch
import torch.optim as optim
import torch. nn.functional as F
import re
import warnings
warnings.filterwarnings('ignore')

2024-04-27 09:21:30.888378: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 09:21:30.888486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 09:21:31.052945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
train_df = pd.read_csv('/kaggle/input/exist-2024-clef/train_data_task2.csv')
dev_df = pd.read_csv('/kaggle/input/exist-2024-clef/dev_data_task2.csv')
test_df = pd.read_csv('/kaggle/input/exist-2024-clef/EXIT2024_tweet_test.csv')

print("----------------- The train dataset is --------------------")
print(train_df.head())
print("----------------- The dev dataset is --------------------")
print(dev_df.head())
print("----------------- The test dataset is --------------------")
print(test_df.head())

----------------- The train dataset is --------------------
       id lang     split                                              tweet  \
0  100001   es  TRAIN_ES  @TheChiflis Ignora al otro, es un capullo.El p...   
1  100002   es  TRAIN_ES  @ultimonomada_ Si comicsgate se parece en algo...   
2  100003   es  TRAIN_ES  @Steven2897 Lee sobre Gamergate, y como eso ha...   
3  100004   es  TRAIN_ES  @Lunariita7 Un retraso social bastante lamenta...   
4  100005   es  TRAIN_ES  @novadragon21 @icep4ck @TvDannyZ Entonces como...   

                                          soft_label hard_label  
0  {'REPORTED': 0.5, 'JUDGEMENTAL': 0.33333333333...   REPORTED  
1  {'NO': 0.833333333333333, 'DIRECT': 0.16666666...         NO  
2  {'NO': 1.0, 'DIRECT': 0.0, 'REPORTED': 0.0, 'J...         NO  
3  {'NO': 0.5, 'DIRECT': 0.16666666666666602, 'RE...        NaN  
4  {'REPORTED': 0.16666666666666602, 'NO': 0.3333...        NaN  
----------------- The dev dataset is --------------------
       id l

In [7]:
device = torch. device('cuda:0'  if  torch.cuda.is_available() else  'cpu')
print(f"computation will run on {device} now")

computation will run on cuda:0 now


# Hard with ALL

In [6]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
    """
    pass the tweet data as a series. do not use apply function
    only preprocesses for replacing @USER and URLS
    """
    # print("i am preprocessing")
    URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
    HANDLE_RE = re.compile(r"@\w+")
    tweets = []
    for t in text:
        t = HANDLE_RE.sub("@USER", t)
        t = URL_RE.sub("HTTPURL", t)
        tweets.append(t)
    return tweets

In [7]:
from sklearn.preprocessing import LabelEncoder

task_encoder = LabelEncoder()

def task_hard_encode(df):
    task_encoder.fit(all_task_hard_labels)
    df['hard_label'] = task_encoder.transform(df['hard_label'])
    return df

def task_hard_decode(df):
    df["hard_label"] = task_encoder.inverse_transform(df["hard_label"])
    return df

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset

In [9]:
og_train1 = train_df.copy()
og_dev1 = dev_df.copy()
og_test1 = test_df.copy()

all_task_hard_labels = pd.concat([og_train1["hard_label"],og_dev1["hard_label"]])
train1_df = task_hard_encode(og_train1)

# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df = train1_df[train1_df['hard_label'] != 4]
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

dev1_df = task_hard_encode(og_dev1)
dev1_df = dev1_df[["tweet","hard_label"]].dropna()
dev1_df = dev1_df[dev1_df['hard_label'] != 4]
dev1_df["tweet"] = simple_preprocess(dev1_df["tweet"])

test1_df = og_test1
test1_df = test1_df[["tweet"]]
# test1_df = test1_df[test1_df['hard_label'] != 2]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

train1 (5496, 2)
<bound method NDFrame.head of                                                   tweet  hard_label
0     @USER Ignora al otro, es un capullo.El problem...           3
1     @USER Si comicsgate se parece en algo a gamerg...           2
2     @USER Lee sobre Gamergate, y como eso ha cambi...           2
5     @USER Aaah sí. Andrew Dobson. El que se dedicó...           2
7     @USER Esta gringa sigue llorando por el gamerg...           0
...                                                 ...         ...
6913  @USER Ma'am if I say that you look like a whor...           0
6915  idk why y’all bitches think having half your a...           0
6916  This has been a part of an experiment with @US...           1
6917  "Take me already" "Not yet. You gotta be ready...           0
6918    @USER why do you look like a whore? /lh HTTPURL           0

[5496 rows x 2 columns]>
test1 (2076, 1)
<bound method NDFrame.head of                                                   tweet
0     @US

In [10]:
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [11]:
# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model/task2/hard",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=0.00017420128933479402,
    weight_decay=6.022010235237979e-05,
    logging_dir="./logs",
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5386 entries, 0 to 5385
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet       5386 non-null   object
 1   hard_label  5386 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 84.3+ KB


In [13]:
from datasets import Dataset

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})


In [18]:
from sklearn.metrics import accuracy_score, f1_score

def compute_f1_score(pred):
    # pred is a tuple (predictions, labels)
    predictions, labels = pred
    # Compute the F1 score
    f1 = f1_score(labels, predictions.argmax(axis=1), average='macro')
    return {"f1_score": f1}

In [15]:
# Define the trainer for each fold
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_f1_score
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss,F1 Score
500,1.102100,1.025356,0.193621
1000,1.080700,1.014190,0.193621


TrainOutput(global_step=1348, training_loss=1.0773036741715156, metrics={'train_runtime': 1671.5515, 'train_samples_per_second': 12.889, 'train_steps_per_second': 0.806, 'total_flos': 5019416241758208.0, 'train_loss': 1.0773036741715156, 'epoch': 4.0})

In [16]:
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score


# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_labels = np.argmax(val_predictions.predictions, axis=1)
val_true_labels = val_dataset["label"]

val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels,average = "weighted")

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.6319663512092534
Validation F1 Score: 0.48944816633603255


In [17]:
# Get predictions on the test set
test_predictions = trainer.predict(test_dataset)
test_pred_labels = np.argmax(test_predictions.predictions, axis=1)
test_pred_probabilities = softmax(test_predictions.predictions).tolist()

In [18]:
def create_results(probabilities, class_labels):
    results = []
    for prob_list in probabilities:
        result_dict = {label: prob for label, prob in zip(class_labels, prob_list)}
        results.append(result_dict)
    return results

test_probs_list = create_results(test_pred_probabilities, task_encoder.classes_)
test_probs_list[:10]

[{'DIRECT': 0.00011659974552458152,
  'JUDGEMENTAL': 3.738896339200437e-05,
  'NO': 0.00029167119646444917,
  'REPORTED': 3.6035708035342395e-05},
 {'DIRECT': 0.00011659975280053914,
  'JUDGEMENTAL': 3.738897430594079e-05,
  'NO': 0.00029167113825678825,
  'REPORTED': 3.6035708035342395e-05},
 {'DIRECT': 0.00011659974552458152,
  'JUDGEMENTAL': 3.738897430594079e-05,
  'NO': 0.00029167113825678825,
  'REPORTED': 3.6035708035342395e-05},
 {'DIRECT': 0.00011659975280053914,
  'JUDGEMENTAL': 3.738896339200437e-05,
  'NO': 0.00029167113825678825,
  'REPORTED': 3.6035708035342395e-05},
 {'DIRECT': 0.00011659974552458152,
  'JUDGEMENTAL': 3.738896339200437e-05,
  'NO': 0.0002916712255682796,
  'REPORTED': 3.6035697121405974e-05},
 {'DIRECT': 0.00011659974552458152,
  'JUDGEMENTAL': 3.738896339200437e-05,
  'NO': 0.0002916711091529578,
  'REPORTED': 3.6035708035342395e-05},
 {'DIRECT': 0.00011659975280053914,
  'JUDGEMENTAL': 3.738897430594079e-05,
  'NO': 0.00029167113825678825,
  'REPORTED'

In [19]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_label"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = decoded_labels[i]
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    print("output generated as json")
    with open('./output/task2_NICA-1.json', 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(test_pred_labels,columns=["hard_label"]), test_probs_list, og_test1[["id"]],og_test1)

decoded labels length: 2076
output generated as json


# Soft with ALL

In [6]:
import re

def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
    """
    pass the tweet data as a series. do not use apply function
    only preprocesses for replacing @USER and URLS
    """
    URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
    HANDLE_RE = re.compile(r"@\w+")
    tweets = []
    for t in text:
        t = HANDLE_RE.sub("@USER", t)
        t = URL_RE.sub("HTTPURL", t)
        tweets.append(t)
    return tweets

def convert_logits_to_list(logits_dict):
    """
    order:
    1. classes are 
    REPORTED, NO, DIRECT, JUDGEMENTAL 
    """
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["REPORTED"], logits_dict["NO"],logits_dict["DIRECT"],logits_dict["JUDGEMENTAL"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"REPORTED": logits_list[0], "NO": logits_list[1],"DIRECT": logits_list[2], "JUDGEMENTAL": logits_list[3]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)

In [7]:
og_train1 = train_df.copy()
og_dev1 = dev_df.copy()
og_test1 = test_df.copy()

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
train1_df = og_train1[["tweet","soft_label"]].dropna()

og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
dev1_df = og_dev1[["tweet","soft_label"]].dropna()


test1_df = og_test1
test1_df = test1_df[["tweet"]]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

train1 (6920, 2)
<bound method NDFrame.head of                                                   tweet  \
0     @USER Ignora al otro, es un capullo.El problem...   
1     @USER Si comicsgate se parece en algo a gamerg...   
2     @USER Lee sobre Gamergate, y como eso ha cambi...   
3     @USER Un retraso social bastante lamentable, g...   
4     @USER @USER @USER Entonces como así es el merc...   
...                                                 ...   
6915  idk why y’all bitches think having half your a...   
6916  This has been a part of an experiment with @US...   
6917  "Take me already" "Not yet. You gotta be ready...   
6918    @USER why do you look like a whore? /lh HTTPURL   
6919  ik when mandy says “you look like a whore” i l...   

                                             soft_label  
0     [0.5, 0.16666666666666602, 0.0, 0.333333333333...  
1     [0.0, 0.833333333333333, 0.16666666666666602, ...  
2                                  [0.0, 1.0, 0.0, 0.0]  
3     [0.333

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [9]:
# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4,ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model/task2/soft",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=0.00017420128933479402,
    weight_decay=6.022010235237979e-05,
    logging_dir="./logs",
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [11]:
import torch.nn as nn
from datasets import Dataset

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset


# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})

In [12]:
def custom_loss_fn(logits, soft_labels):
    probs = F.softmax(logits, dim=1)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [13]:
# Define the trainer for each fold
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss
500,19.906200,39.279976
1000,19.895000,39.279976
1500,19.895000,39.279976


TrainOutput(global_step=1692, training_loss=19.89782455119681, metrics={'train_runtime': 2099.9521, 'train_samples_per_second': 12.884, 'train_steps_per_second': 0.806, 'total_flos': 6303626338516992.0, 'train_loss': 19.89782455119681, 'epoch': 4.0})

In [15]:
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score
# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_logit_labels = val_predictions.predictions
val_pred_probabilities = softmax(val_pred_logit_labels, axis=1)

val_pred_hard_labels = np.argmax(val_pred_probabilities, axis=1)
val_true_labels = val_dataset["label"]
val_true_hard_labels = np.argmax(val_dataset["label"],axis=1)

# Calculate evaluation metrics for each fold
val_accuracy = accuracy_score(val_true_hard_labels, val_pred_hard_labels)
val_f1_score = f1_score(val_true_hard_labels, val_pred_hard_labels,average='weighted')

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.6381909547738693
Validation F1 Score: 0.4972408052532601


In [16]:
# Get predictions on the test set
test_predictions1 = trainer.predict(test_dataset)
test_pred_labels1 = np.argmax(test_predictions1.predictions, axis=1)
test_pred_probabilities1 = softmax(test_predictions1.predictions).tolist()

In [17]:
def create_results(probabilities, class_labels):
    results = []
    for prob_list in probabilities:
        result_dict = {label: prob for label, prob in zip(class_labels, prob_list)}
        results.append(result_dict)
    return results

classes_labels = ["REPORTED", "NO", "DIRECT", "JUDGEMENTAL"]

test_probs_list1 = create_results(test_pred_probabilities1, classes_labels)
test_probs_list1

[{'REPORTED': 1.801458704431269e-10,
  'NO': 0.0004816946748178452,
  'DIRECT': 1.1463443388981531e-10,
  'JUDGEMENTAL': 2.0012283497017336e-10},
 {'REPORTED': 1.8014535696497802e-10,
  'NO': 0.000481695169582963,
  'DIRECT': 1.1463432286751285e-10,
  'JUDGEMENTAL': 2.0012265455893186e-10},
 {'REPORTED': 1.8014467695337544e-10,
  'NO': 0.0004816946748178452,
  'DIRECT': 1.1463443388981531e-10,
  'JUDGEMENTAL': 2.0012229373644885e-10},
 {'REPORTED': 1.801458704431269e-10,
  'NO': 0.000481695169582963,
  'DIRECT': 1.1463420490631648e-10,
  'JUDGEMENTAL': 2.0012305701477828e-10},
 {'REPORTED': 1.8014346958583616e-10,
  'NO': 0.000481696508359164,
  'DIRECT': 1.1463443388981531e-10,
  'JUDGEMENTAL': 2.0012169699157312e-10},
 {'REPORTED': 1.8014535696497802e-10,
  'NO': 0.0004816942091565579,
  'DIRECT': 1.1463420490631648e-10,
  'JUDGEMENTAL': 2.0012265455893186e-10},
 {'REPORTED': 1.8014828517820547e-10,
  'NO': 0.00048169289948418736,
  'DIRECT': 1.1463410082290793e-10,
  'JUDGEMENTAL': 

In [23]:
from sklearn.preprocessing import LabelEncoder
task_encoder = LabelEncoder()

def task_hard_encode(df):
    task_encoder.fit(all_task_hard_labels)
    df['hard_label'] = task_encoder.transform(df['hard_label'])
    return df

def task_hard_decode(df):
    df["hard_label"] = task_encoder.inverse_transform(df["hard_label"])
    return df

In [24]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_label"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = decoded_labels[i]
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "./output/task2_NICA-2.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(test_pred_labels1,columns=["hard_label"]), test_probs_list1, og_test1[["id"]],og_test1)

NotFittedError: This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.